Code taken from the official BerTopic github/site: https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html \
Code for reducing outliers based on probability: https://github.com/MaartenGr/BERTopic/issues/529 \
Covert output to features: https://stackoverflow.com/questions/73768683/how-to-get-topic-probs-matrix-in-bertopic-modeling

Visual Novel BerTopic

In [1]:
pip install bertopic


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 62.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=818347 sha256=0fbb1506d2300ed38feb8251e317bd12626243b0cb59ecc8ab3b7748921c0708
  Stored in directory: /root/.cache/pip/wheels/dc/52/e3/6c6b60b126b4d5c4370cb5ac071b82950f91649d62d72f7f56
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e6bfaa1abe3f535b7138323086f4a7532d006a3571109d9e4c6afb3b3ed10c06
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully 

In [2]:
from bertopic import BERTopic
import pandas
import numpy
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [3]:
additional = ["game", "play"]
stop = ENGLISH_STOP_WORDS.union(additional)

In [4]:
#loading the dataset
df = pandas.read_csv("/kaggle/input/dungeon-final-bert/dungeon_final_Bert.csv")

In [7]:
df.head()

,app_id,app_name,review_text,review_score,App ID,Genre,Owners,Price,Initial Price,Required Age,Tags
0,113200,The Binding of Isaac,"After playing Super meat boy, I started lookin...",0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
1,113200,The Binding of Isaac,As of writing this I am 500+ hours in and I ha...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
2,113200,The Binding of Isaac,The Binding of Isaac is a small game with very...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
3,113200,The Binding of Isaac,This is pretty much a roguelike dungeon crawle...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
4,113200,The Binding of Isaac,"A stream-lined rouge-like, filled with morbid ...",0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."


In [ ]:
df

,app_id,app_name,review_text,review_score,App ID,Genre,Owners,Price,Initial Price,Required Age,Tags
0,113200,The Binding of Isaac,"After playing Super meat boy, I started lookin...",0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
1,113200,The Binding of Isaac,As of writing this I am 500+ hours in and I ha...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
2,113200,The Binding of Isaac,The Binding of Isaac is a small game with very...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
3,113200,The Binding of Isaac,This is pretty much a roguelike dungeon crawle...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
4,113200,The Binding of Isaac,"A stream-lined rouge-like, filled with morbid ...",0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
...,...,...,...,...,...,...,...,...,...,...,...
23560,99900,Spiral Knights,Early game is pretty fun and fairly easy to ad...,1,99900,"Action, Adventure, Casual, Free to Play, Indie...","2,000,000 .. 5,000,000",0,0,0,"Free to Play: 1167, Massively Multiplayer: 644..."
23561,99900,Spiral Knights,"Spiral Knights seems great, but lacks in plen...",1,99900,"Action, Adventure, Casual, Free to Play, Indie...","2,000,000 .. 5,000,000",0,0,0,"Free to Play: 1167, Massively Multiplayer: 644..."
23562,99900,Spiral Knights,I first said this game was super cute and all ...,1,99900,"Action, Adventure, Casual, Free to Play, Indie...","2,000,000 .. 5,000,000",0,0,0,"Free to Play: 1167, Massively Multiplayer: 644..."
23563,99900,Spiral Knights,"Having played this game for around 8,000 Hours...",0,99900,"Action, Adventure, Casual, Free to Play, Indie...","2,000,000 .. 5,000,000",0,0,0,"Free to Play: 1167, Massively Multiplayer: 644..."


In [7]:
review = df["review_text"].values

In [5]:
#preperation
embedding_model = SentenceTransformer("all-mpnet-base-v2")
UMAP_model = UMAP(random_state = 101)
hdbscan_model = HDBSCAN( metric='euclidean', prediction_data = True, min_samples =3, min_cluster_size = 20 )
vectorizer_model = CountVectorizer(min_df = 5, stop_words=list(stop))
ctfidf_model = ClassTfidfTransformer()

In [6]:
#making topic model
topic_model = BERTopic(embedding_model=embedding_model, umap_model= UMAP_model, hdbscan_model= hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, calculate_probabilities=True, nr_topics=121)

In [8]:
topics, probs = topic_model.fit_transform(review) 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [30]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,313,-1_zone_dont_let_dungeon
1,0,1522,0_diablo_exile_path_poe
2,1,812,1_rebirth_original_isaac_binding
3,2,1344,2_isaac_binding_mother_items
4,3,677,3_torchlight_diablo_ii_pet
...,...,...,...
116,115,22,115_kingdoms_falcom_shadows_heretic
117,116,25,116_killed_hours_speed_team
118,117,25,117_step_level_boss_really
119,118,34,118_family_hamlet_die_mobs


In [10]:
# Use the "c-TF-IDF" strategy with a threshold
probability_threshold = 0.01
new_topics = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in probs]

In [11]:
topic_model.update_topics(review, topics=new_topics, vectorizer_model=vectorizer_model)

In [32]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,313,-1_zone_dont_let_dungeon
1,0,1522,0_diablo_exile_path_poe
2,1,812,1_rebirth_original_isaac_binding
3,2,1344,2_isaac_binding_mother_items
4,3,677,3_torchlight_diablo_ii_pet
...,...,...,...
116,115,22,115_kingdoms_falcom_shadows_heretic
117,116,25,116_killed_hours_speed_team
118,117,25,117_step_level_boss_really
119,118,34,118_family_hamlet_die_mobs


In [42]:
topic_model.get_topic(98)

[('sugar', 0.11357399135116464),
 ('pie', 0.09678664478099878),
 ('cup', 0.0806181501061707),
 ('butter', 0.07940090094531019),
 ('salt', 0.06805226701629905),
 ('heat', 0.06611975111904551),
 ('dough', 0.06596404088637442),
 ('pan', 0.0656375216180304),
 ('bowl', 0.061929470870277864),
 ('oven', 0.06082602091068779)]

In [36]:
#getting output features
output_features = pandas.DataFrame(probs)
output_features["sentiment"] = df["review_score"]


In [37]:
output_features

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,sentiment
0,1.554550e-02,1.081630e-01,5.663138e-02,2.333239e-03,3.596339e-02,2.098619e-03,2.364012e-02,5.254515e-03,6.838805e-03,6.299661e-03,...,1.165053e-03,4.567182e-03,3.278959e-03,2.556096e-03,2.139480e-03,1.673649e-03,2.630820e-03,3.593357e-03,1.478194e-03,0
1,6.519406e-03,8.169974e-03,2.608046e-02,9.225114e-04,3.512590e-02,1.206870e-03,2.487019e-02,2.956705e-03,5.401715e-03,4.332557e-03,...,7.102972e-04,5.733551e-03,4.230669e-03,2.152340e-03,1.521066e-03,1.196298e-03,2.406235e-03,3.932182e-03,4.843089e-04,0
2,8.509663e-307,1.000000e+00,3.784100e-306,1.556598e-307,2.065353e-306,1.298450e-307,1.291607e-306,2.963634e-307,4.050248e-307,3.865164e-307,...,7.147302e-308,2.582192e-307,1.791893e-307,1.400413e-307,1.195653e-307,1.024952e-307,1.576184e-307,1.925494e-307,9.259370e-308,0
3,1.051417e-306,2.016642e-306,9.047224e-306,1.369693e-307,4.872387e-306,1.601557e-307,2.919845e-306,4.303837e-307,6.876297e-307,5.487640e-307,...,9.257109e-308,7.845563e-307,4.551687e-307,2.742387e-307,2.091530e-307,1.497740e-307,2.953929e-307,4.720487e-307,7.251976e-308,0
4,1.707980e-306,5.874200e-307,1.161093e-306,8.777461e-308,1.801795e-306,1.510206e-307,3.794988e-306,7.456014e-307,5.806310e-307,4.831227e-307,...,9.384838e-308,2.244154e-307,4.294892e-307,4.862144e-307,2.677420e-307,1.300161e-307,1.668917e-307,7.690572e-307,6.718824e-308,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23560,7.206797e-03,4.045883e-03,8.242443e-03,7.917606e-04,1.409944e-02,1.749929e-03,3.167182e-02,1.859028e-02,6.202524e-03,5.474443e-03,...,1.161423e-03,1.671172e-03,3.111742e-03,6.347289e-03,4.234082e-03,1.459434e-03,1.529516e-03,3.492920e-03,6.021542e-04,1
23561,3.115431e-04,1.818786e-04,3.693197e-04,2.805660e-05,6.146754e-04,6.753109e-05,1.191088e-03,1.273958e-03,2.812027e-04,2.883714e-04,...,5.278478e-05,7.177979e-05,1.207654e-04,2.146819e-04,2.141143e-04,7.620899e-05,7.412592e-05,1.261484e-04,2.136162e-05,1
23562,8.326945e-307,5.434685e-307,1.179855e-306,9.456703e-308,2.204470e-306,1.831498e-307,1.000000e+00,6.626891e-307,1.082098e-306,6.737607e-307,...,1.186582e-307,2.706973e-307,1.265684e-306,1.189955e-306,4.832029e-307,1.863678e-307,2.391894e-307,1.040852e-306,6.045012e-308,1
23563,3.114336e-04,1.817410e-04,3.690426e-04,2.801808e-05,6.141782e-04,6.737864e-05,1.192123e-03,1.269790e-03,2.810003e-04,2.878924e-04,...,5.265604e-05,7.173171e-05,1.208083e-04,2.151753e-04,2.140637e-04,7.609164e-05,7.406042e-05,1.262463e-04,2.132820e-05,0


In [38]:
output_features.to_csv("Features_Dungeon_Bert_reduction.csv")

In [ ]:
#saving model and features
topic_model.save("Dungeon_model_Bert")